<a href="https://colab.research.google.com/github/Layondf/Projeto-Imers-o-Aula-4-Alura-Google/blob/main/Leitura_de_Chapas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Leitura da Cena**

In [94]:
import os
import hashlib
import datetime
import pytz
import google.generativeai as genai
from google.colab import userdata
userdata.get('SECRET_KEY')
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

def salvar_texto_gerado(texto, nome_arquivo):
    with open(nome_arquivo, 'w') as arquivo:
        arquivo.write(texto)

def obter_ultimo_arquivo(path):
    arquivos = [os.path.join(path, f) for f in os.listdir(path) if f.endswith('.txt')]
    if arquivos:
        return max(arquivos, key=os.path.getmtime)
    return None

configuracao_geracao = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

configuracao_seguranca = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

modelo = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=configuracao_geracao,
                              safety_settings=configuracao_seguranca)

partes_prompt = [
  "Reconheça as posições em que A,B ou C representa Linha e 1,2 e 3 Representa Coluna e leia os números. Nunca escrever description",
  "input: ",
  genai.upload_file("/content/drive/MyDrive/Colab Notebooks/Modelos de Aprendizado/Cena1.jpeg"),
  "output: description A1 3023710817\nA2 3023710816\nA3 3023710834\nB1 3023710836\nB2 3023710813\nB3 3023710814\nC1 3023710824\nC2 3023710810\nC3 3023710828",
  "input: ",
  genai.upload_file("/content/drive/MyDrive/Colab Notebooks/Modelos de Aprendizado/Cena2.jpeg"),
  "output: description A1 3023710817\nA2 3023710816\nA3 3023710834\nB1 3023710818\nB2 3023710813\nB3 3023710814\nC1 3023710815\nC2 3023710810\nC3 3023710828",
  "input: ",
  genai.upload_file("/content/drive/MyDrive/Colab Notebooks/Modelos de Aprendizado/Cena4.jpeg"),
  "output: description A1 3023710834\nA2 3023710816\nA3 3023710810\nB1 3023710814\nB2 0\nB3 3023710824\nC1 3023710815\nC2 3023710836\nC3 3023710817",
  "input: ",
  genai.upload_file("/content/drive/MyDrive/Colab Notebooks/Teste1.jpeg"),
  "output: description ",
]

#
resposta = modelo.generate_content(partes_prompt)
print(resposta.text)

# Pasta onde os resultados serão salvos
pasta_resultados = '/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/'
nome_arquivo_salvo = 'syslog.txt'
caminho_arquivo_salvo = os.path.join(pasta_resultados, nome_arquivo_salvo)

# Salvar o texto gerado no arquivo de log
salvar_texto_gerado_com_datahora(resposta.text, caminho_arquivo_salvo)
print(f"Texto gerado salvo em: {caminho_arquivo_salvo}")


A1 3023710834
A2 3023710811
A3 3023710810
B1 3023710814
B2 3023710828
B3 3023710824
C1 3023710835
C2 3023710836
C3 3023710817 

Texto gerado salvo em: /content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt


## **Modo Generativo**

In [ ]:
from pathlib import Path
import hashlib
import google.generativeai as genai

userdata.get('SECRET_KEY')
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

system_instruction = "Este texto é um registro de movimentação de chapas. As letras A, B e C representam as linhas, enquanto os números 1, 2 e 3 representam as colunas. Para analisar o texto e determinar se foram adicionadas ou retiradas chapas, siga estas orientações:\n\nSe houver uma mudança no número na célula, sem que ele vire zero, significa que uma chapa foi adicionada. Se o número mudar sem se tornar zero, isso indica que foi adicionada uma chapa acima, resultando em duas ou mais chapas na célula.\n\nSe uma célula não contiver nenhum número, isso significa que não há mais chapas naquela célula.\n\nSe o número mudar sem chegar a zero, significa que foram adicionadas chapas. Quando todas as chapas forem removidas, o número voltará a zero. Não responder nada que não seja baseado no arquivo TXT"

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

def extract_text_file(pathname: str) -> list[str]:
  parts = [f"--- START OF FILE ${pathname} ---"]
  with open(pathname, 'r') as file:
    parts.extend(file.readlines())
  return parts

# Carregar o arquivo syslog.txt
file_path = "/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt"
file_content = extract_text_file(file_path)

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": file_content
  }
])

# Looping para receber perguntas do usuário
while True:
  # Solicitar a pergunta ao usuário
  user_question = input("Digite sua pergunta sobre o arquivo syslog.txt (ou digite 'fim' para sair): ")

  # Encerrar o looping se o usuário digitar 'fim'
  if user_question.lower() == 'fim':
    break

  # Enviar a pergunta ao modelo
  convo.send_message(user_question)

  # Imprimir a resposta do modelo
  print(convo.last.text)

## **Modo Embedding em Evolução**

In [100]:
pip install -U -q google.generativeai

In [101]:
import textwrap
import numpy as np
import pandas as pd

import google.generativeai as genai
import google.ai.generativelanguage as glm

# Used to securely store your API key
from google.colab import userdata

from IPython.display import Markdown

In [102]:
userdata.get('SECRET_KEY')
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

In [103]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [105]:
title = "The next generation of AI for developers and Google Workspace"
sample_text = ("/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt")

model = 'models/embedding-001'
embedding = genai.embed_content(model=model,
                                content=sample_text,
                                task_type="retrieval_document",
                                title=title)

print(embedding)

{'embedding': [0.06490323, 0.0020456193, -0.045785513, -0.04981185, 0.08248697, 0.028622484, -0.0047157644, 0.0033530628, 0.012299381, 0.064134434, -0.0027007638, 0.0075182673, -0.04782076, -0.0067438036, 0.029550051, -0.05606037, 0.0018542964, 0.0085241, -0.047573846, 0.004356591, 0.033939917, 0.004386333, -0.018151702, -0.02710876, -0.0037827964, -0.021774173, -0.016282007, -0.07001061, -0.0048251203, 0.022249281, -0.04848326, 0.038759332, -0.04927008, 0.019387834, -0.04836948, -0.0069651655, 0.0039500524, 0.02153253, 0.003767384, 0.01574518, -0.03786213, -0.06412854, 0.015698897, 0.019142913, 0.0058201505, 0.013146148, 0.030006178, 0.012369762, -0.0071903085, -0.086549595, 0.019238183, 0.042855877, 0.036476567, -0.023684204, -0.028512731, -0.00026227313, 0.053711772, -0.010078259, -0.0068280315, 0.0011383155, 0.007908845, 0.06140848, -0.063358694, 0.075211495, -0.03747509, -0.042146597, -0.0438655, -0.016959969, 0.023123328, 0.011806416, 0.024806917, -0.017245209, 0.03687641, -0.040

In [106]:
DOCUMENT1 = {
    "title": "Syslog",
    "content": "/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt"}


documents = [DOCUMENT1]

In [107]:
df = pd.DataFrame(documents)
df.columns = ['Title', 'Text']
df

,Title,Text
0,Syslog,/content/drive/MyDrive/Colab Notebooks/Logs Mo...


In [108]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
def embed_fn(title, text):
  return genai.embed_content(model=model,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

df['Embeddings'] = df.apply(lambda row: embed_fn(row['Title'], row['Text']), axis=1)
df

,Title,Text,Embeddings
0,Syslog,/content/drive/MyDrive/Colab Notebooks/Logs Mo...,"[0.06969882, -0.0075141788, -0.061609257, -0.0..."


In [109]:
query = "Qual chapa saiu?"
model = 'models/embedding-001'

request = genai.embed_content(model=model,
                              content=query,
                              task_type="retrieval_query")

In [110]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding["embedding"])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

In [111]:
passage = find_best_passage(query, df)
passage

'/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt'

In [112]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [113]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'Qual chapa saiu?'
  PASSAGE: '/content/drive/MyDrive/Colab Notebooks/Logs Movimentação/syslog.txt'

    ANSWER:



In [114]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [116]:
model = genai.GenerativeModel('gemini-1.0-pro-latest')
answer = model.generate_content(prompt)

In [117]:
Markdown(answer.text)

I'm sorry, but there is no mention of 'Qual chapa saiu?' in the reference passage you provided.